In [ ]:
import glob
import os
import io
import pymap3d

import cartopy.feature as cf
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import shapely
import thalassa
import xarray as xr

import searvey

import pyposeidon
import pyposeidon.meteo as pmeteo
import pyposeidon.dem as pdem
import pyposeidon.boundary as pbound
import pyposeidon.mesh as pmesh
import pyposeidon.model as pmodel
from pyposeidon.utils import cast

import pyposeidon.utils.hplot as hplot
import pyposeidon.utils.pplot as pplot

hv.extension("bokeh")

!mkdir -p data/iceland

# Geometry

In [ ]:
lon_min = -28.0
lon_max = -11.1
lat_min =  62.0
lat_max =  68.0

bbox = shapely.box(lon_min, lat_min, lon_max, lat_max)
geometry = dict(lon_min=lon_min, lon_max=lon_max, lat_min=lat_min, lat_max=lat_max)

# Coastlines

In [ ]:
OSM_FOLDER = "/home/tomsail/work/python/seareport_org/coastlines/raw/osm/land-polygons-complete-4326"

In [ ]:
coastlines = gpd.read_file(OSM_FOLDER + "/land_polygons.shp", bbox=bbox)
coastlines

# Boundaries

In [ ]:
boundary = pbound.Boundary(geometry=geometry, coastlines=coastlines)
boundary.contours.head()
len(boundary.contours)

In [ ]:
boundary.show()

In [ ]:
# boundary.contours.hvplot(geo=True, tiles=True, frame_height=500)

# DEM

In [ ]:
# url = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/srtm30plus"
url = "data/iceland.nc"
dem = pdem.Dem(dem_source=url, **geometry)
dem.Dataset.load()

In [ ]:
dem.Dataset.to_netcdf("./data/iceland/dem.nc")

# Mesh

In [ ]:
mesh = pmesh.set(
    mesh_generator='oceanmesh',
    bgmesh = "om",
    dem_source="./data/iceland/dem.nc",
    type='tri2d',
    geometry=geometry,
    coastlines=coastlines,
    grad = 0.15,
    bathy_gradient= True,
    resolution_min=0.02,
    resolution_max=1.50,
    alpha_wavelength= 100,  # number of element to resolve WL
    alpha_slope= 10,  # number of element to resolve bathy gradient
)
mesh.Dataset.dims

In [ ]:
mesh.Dataset.pplot.mesh()

In [ ]:
mesh.to_file('./data/iceland/hgrid.gr3')
!head ./data/iceland/hgrid.gr3

### Fix bathy

In [ ]:
#define in a dictionary the properties of the model..
model_parameters = {
    "solver_name": "telemac",
    "rpath": "./data/iceland/",
    "dem_source": "./data/iceland/dem.nc",
    "mesh_file": "./data/iceland/hgrid.gr3",
    "update": ["dem"], #set which component should be updated  (meteo,dem,model)
    "start_date": "2018-10-01T00:00:00",
    "time_frame": "1d",
    "global": False,
}
model_parameters

In [ ]:
model = pmodel.set(**model_parameters)
model.create()
model.mesh.to_file('./data/iceland/hgrid.gr3')
!head ./data/iceland/hgrid.gr3

In [ ]:
def parse_hgrid_nodes(path: os.PathLike[str] | str) -> pd.DataFrame:
    with open(path, "rb") as fd:
        _ = fd.readline()
        _, no_points = map(int, fd.readline().strip().split(b" "))
        content = io.BytesIO(b''.join(next(fd) for _ in range(no_points)))
        nodes = pd.read_csv(
            content,
            engine="pyarrow",
            sep="\t",
            header=None,
            names=["lon", "lat", "depth"],
            index_col=0
        )
    nodes = nodes.reset_index(drop=True)
    return nodes
    
def parse_hgrid_elements3(path: os.PathLike[str] | str) -> pd.DataFrame:
    with open(path, "rb") as fd:
        _ = fd.readline()
        no_elements, no_points = map(int, fd.readline().strip().split(b" "))
        for _ in range(no_points):
            next(fd) 
        content = io.BytesIO(b''.join(next(fd) for _ in range(no_elements)))
        elements = pd.read_csv(
            content,
            engine="pyarrow",
            sep="\t",
            header=None,
            names=["no_nodes", "n1", "n2", "n3"],
            index_col=0
        )
    elements = elements.assign(
        n1=elements.n1 - 1,
        n2=elements.n2 - 1,
        n3=elements.n3 - 1,
    ).reset_index(drop=True)
    return elements

def get_skews_and_base_cfls(lons, lats, depths) -> np.ndarray:
    # The shape of each one of the input arrays needs to be (3, <no_triangles>)
    #ell = pymap3d.Ellipsoid.from_name("wgs84")
    ell = pymap3d.Ellipsoid(6378206.4, 6378206.4, "schism", "schism")
    local_x, local_y, _ = pymap3d.geodetic2enu(lats, lons, depths, lats[0], lons[0], depths[0], ell=ell)
    areas = (local_x[1] * local_y[2] - local_x[2] * local_y[1]) * 0.5
    rhos = np.sqrt(areas / np.pi)
    max_sides = np.maximum(
        np.sqrt(local_x[1] ** 2 + local_y[1] ** 2),
        np.sqrt(local_x[2] ** 2 + local_y[2] ** 2),
        np.sqrt((local_x[2] - local_x[1]) ** 2 + (local_y[2] - local_y[1]) ** 2),
    )
    skews = max_sides / rhos
    base_cfls = np.sqrt(9.81 * np.maximum(0.1, depths.mean(axis=0))) / rhos / 2
    return skews, base_cfls

def get_skews_and_base_cfls_from_path(path: os.PathLike[str] | str) -> np.ndarray:
    nodes = parse_hgrid_nodes(path)
    elements = parse_hgrid_elements3(path)
    tri = elements[["n1", "n2", "n3"]].values
    lons = nodes.lon.values[tri].T
    lats = nodes.lat.values[tri].T
    depths = nodes.depth.values[tri].T
    skews, base_cfls = get_skews_and_base_cfls(lons=lons, lats=lats, depths=depths)
    return skews, base_cfls
    

In [ ]:
skews, base_cfls = get_skews_and_base_cfls_from_path("./data/iceland/hgrid.gr3")
CFL_THRESHOLD = 0.4
print(f"elements violating CFL threshold < {CFL_THRESHOLD}")
print("time            N         %")
for dt in (1, 50, 75, 100, 120, 150, 200, 300, 400, 600, 900, 1200, 1800, 3600):
    violations = (base_cfls * dt < CFL_THRESHOLD).sum()
    print(f"{dt:>4d} {violations:>12d} {violations / len(base_cfls) * 100:>8.2f}%")
    

In [ ]:
pd.DataFrame({"skew": skews}).describe([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.995, 0.999])

In [ ]:
def get_meta() -> gpd.GeoDataFrame:
    meta_web = searvey.get_ioc_stations().drop(columns=["lon", "lat"])
    meta_api = (
        pd.read_json(
            "http://www.ioc-sealevelmonitoring.org/service.php?query=stationlist&showall=all"
        )
        .drop_duplicates()
        .drop(columns=["lon", "lat"])
        .rename(columns={"Code": "ioc_code", "Lon": "longitude", "Lat": "latitude"})
    )
    merged = pd.merge(
        meta_web,
        meta_api[["ioc_code", "longitude", "latitude"]].drop_duplicates(),
        on=["ioc_code"],
    )
    updated = merged.assign(
        geometry=gpd.points_from_xy(merged.longitude, merged.latitude, crs="EPSG:4326")
    )
    return updated

ioc_ = get_meta()
ioc_[bbox.contains(ioc_.geometry)].to_csv('data/iceland/stations.csv')

In [ ]:
#define in a dictionary the properties of the model..
model_parameters = {
    "solver_name": "telemac",
    "tag": "telemac2d",
    "rpath": "./data/iceland/20181001",
    "mesh_file": "./data/iceland/hgrid.gr3",
    "update": ["all"], #set which component should be updated  (meteo,dem,model)
    "meteo_source": glob.glob("data/uvp_*.grib"),
    "meteo_merge": "last",  # combine meteo
    "meteo_combine_by": "nested",
    "meteo_xr_kwargs": {"concat_dim": "step"},
    "start_date": "2018-10-01T00:00:00",
    "time_frame": "2d",
    "obs": "data/iceland/stations.csv",
    "monitor": True,
    "parameters": {
        "dt": 100
    }
}
model_parameters

## run days 1&2

In [ ]:
a = pmodel.set(**model_parameters)
a.create()
# IMPORTANT! Here, for a simple surge application, 
# we will need close all boundaries, otherwise the 
# model will run out of water
a.mesh.Dataset.type[:] = 'closed' # it will create a cli file with all boundaries closed (this can be done only once)
# we also need to drop some meteo variables, it is necesarry for zarr export
a.output(**{"global": False})
a.set_obs()
a.save() # saves the json model file
a.run() # runs the model

## run days 3&4 from hotstart

In [ ]:
# restart model
prev_ = pd.Timestamp('2018-10-01')
next_ = pd.Timestamp('2018-10-03')
end_ = pd.Timestamp('2018-10-05')
ppath = os.path.join('data/iceland', prev_.strftime("%Y%m%d"))
npath = os.path.join('data/iceland', next_.strftime("%Y%m%d"))
m = pyposeidon.model.read(os.path.join(ppath, "telemac2d_model.json"))
meteo = pmeteo.Meteo(glob.glob("data/uvp_*.grib"),meteo_merge= "last", meteo_combine_by= "nested", meteo_xr_kwargs= {"concat_dim": "step"},)
rs = cast.set(
    solver_name="telemac",
    model=m,
    ppath=ppath,  # old path
    cpath=npath,  # new path
    meteo=meteo.Dataset.sel(time=slice(next_, end_)).compute(),
    sdate=next_,  # new start date
    end_date=end_,  # new end date
    start=next_,  # start
    copy=True,
)
b = rs.run(execute=False)

In [ ]:
b.run()

## run 4 days model - check

In [ ]:
#define in a dictionary the properties of the model..
model_parameters = {
    "solver_name": "telemac",
    "tag": "telemac2d",
    "rpath": "./data/iceland/20181001-04",
    "mesh_file": "./data/iceland/hgrid.gr3",
    "update": ["all"], #set which component should be updated  (meteo,dem,model)
    "meteo_source": glob.glob("data/uvp_*.grib"),
    "meteo_merge": "last",  # combine meteo
    "meteo_combine_by": "nested",
    "meteo_xr_kwargs": {"concat_dim": "step"},
    "start_date": "2018-10-01T00:00:00",
    "time_frame": "4d",
    "global": False,
    "obs": "data/iceland/stations.csv",
    "monitor": True,
    "parameters": {
        "dt": 100
    }
}
model_parameters

In [ ]:
c = pmodel.set(**model_parameters)
c.create()
# IMPORTANT! Here, for a simple surge application, 
# we will need close all boundaries, otherwise the 
# model will run out of water
c.mesh.Dataset.type[:] = 'closed' # it will create a cli file with all boundaries closed (this can be done only once)
# we also need to drop some meteo variables, it is necesarry for zarr export
c.meteo.Dataset = c.meteo.Dataset.compute()
c.output()
c.set_obs()
c.save() # saves the json model file
c.run() # runs the model

## compare results

In [ ]:
res_2days = xr.open_dataset("data/iceland/20181001-04/results_2D.slf")
res_day1 = xr.open_dataset("data/iceland/20181001/results_2D.slf")
res_day2 = xr.open_dataset("data/iceland/20181003/results_2D.slf")

In [ ]:
node = 15000
p1 = res_day1.S.isel(node = node).hvplot(label = "Day 1&2")
p2 = res_day2.S.isel(node = node).hvplot(label = "Day 3&4")
p3 = res_2days.S.isel(node = node).hvplot(label = "4 Days", line_dash='dashed')
p1 * p2 * p3

## compare with observations

In [ ]:
res_2days = xr.open_dataset("data/iceland/20181001-04/results_1D.slf")
res_day1 = xr.open_dataset("data/iceland/20181001/results_1D.slf")
res_day2 = xr.open_dataset("data/iceland/20181003/results_1D.slf")
res_day2

In [ ]:
stations = pd.read_csv("data/iceland/stations.csv")
stations

In [ ]:
from searvey import ioc
data = ioc.get_ioc_station_data('reyk', endtime="2018-10-05", period=30)
data.index = data['time']
data = data.drop(columns=['time'])
data

In [ ]:
# detide 
from analysea.tide import detide
surge = detide(data["prs"],lat = 64.15)
surge

In [ ]:
p1 = res_day1.S.isel(node = 0).hvplot(label = "Day 1")
p2 = res_day2.S.isel(node = 0).hvplot(label = "Day 2")
p3 = res_2days.S.isel(node = 0).hvplot(label = "2 Days", line_dash='dashed')
obs_ = surge.loc["2018-10-01":"2018-10-05"].hvplot(label = "Observations", color = 'k', line_dash='dotted')
p1 * p2  * p3 * obs_